## Обработка признаков, перед подачей в модель

## После обработки values и dates удалим эти столбцы

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Чтение данных из parquet файла
df_test = pd.read_parquet('/home/jupyter/datasphere/vk/test.parquet')

# Преобразование строковых дат в формат datetime
df_test['dates'] = df_test['dates'].apply(pd.to_datetime)

# Извлечение временных признаков
df_test['start_date'] = df_test['dates'].apply(lambda x: x[0])  # Первая дата
df_test['end_date'] = df_test['dates'].apply(lambda x: x[-1])   # Последняя дата
df_test['duration_days'] = df_test['end_date'] - df_test['start_date']  # Длительность периода

# Извлечение года и месяца из первой даты
df_test['start_year'] = df_test['start_date'].dt.year
df_test['start_month'] = df_test['start_date'].dt.month

# Вычисление среднего временного интервала между датами
df_test['mean_interval_days'] = df_test['dates'].apply(
    lambda x: (x[-1] - x[0]).days / len(x) if len(x) > 1 else 0
)

# Агрегация значений из списка 'values'
df_test['median'] = df_test['values'].apply(np.median)  # Медиана значений
df_test['mean'] = df_test['values'].apply(np.mean)      # Среднее значение
df_test['std'] = df_test['values'].apply(np.std)        # Стандартное отклонение
df_test['min'] = df_test['values'].apply(np.min)        # Минимальное значение
df_test['max'] = df_test['values'].apply(np.max)        # Максимальное значение
df_test['last_value'] = df_test['values'].apply(lambda x: x[-1])  # Последнее значение в списке
df_test['sum'] = df_test['values'].apply(np.sum)        # Сумма значений

# Коэффициент вариации (отношение стандартного отклонения к среднему)
df_test['coef_of_variation'] = df_test['std'] / df_test['mean']

# Экспоненциальное скользящее среднее (EMA) с длиной окна 3
df_test['ema'] = df_test['values'].apply(lambda x: pd.Series(x).ewm(span=3).mean().iloc[-1])

# Линейный тренд (наклон линии регрессии) по значениям
df_test['trend_slope'] = df_test['values'].apply(lambda x: linregress(range(len(x)), x)[0])

# Длина списка значений
df_test['length'] = df_test['values'].apply(len)

# Коэффициент эксцесса (kurtosis), характеризующий форму распределения значений
df_test['kurtosis'] = df_test['values'].apply(lambda x: pd.Series(x).kurtosis())


## Удалим dates и values

In [ ]:
df_test.drop(columns = ['dates','values'], inplace = True)

## Предсказание на test

In [5]:
from catboost import CatBoostClassifier

# Загрузка модели из файла
loaded_model = CatBoostClassifier()  # или CatBoostRegressor
loaded_model.load_model('catboost_model.cbm')

# Использование загруженной модели для предсказаний
y_pred = loaded_model.predict_proba(df_test)


##сохранение предсказаний в csv файл
from IPython.display import FileLink

res = pd.DataFrame(y_pred)
res = pd.concat([df_test['id'],res], axis = 1)
result = res.rename(columns = {0:'score'})
FileLink('result.csv')

/home/jupyter/datasphere/vk/result.csv